In [1]:
#pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [6]:
def get_availability(service, attendees, start_time, end_time):
    body = {
        "timeMin": start_time,
        "timeMax": end_time,
        "timeZone": "UTC",
        "items": [{"id": attendee} for attendee in attendees]
    }
    eventsResult = service.freebusy().query(body=body).execute()
    return eventsResult['calendars']

Event created: https://www.google.com/calendar/event?eid=cnZuZTZqaHVmbGJjZWVudjg4amplZTcyODAgZ3J1cG8yZGVzYWZpb2RldHJpcHVAbQ


In [ ]:
def find_best_time(service, attendees, date):
    possible_times = [(datetime.datetime(date.year, date.month, date.day, hour, 0), 
                       datetime.datetime(date.year, date.month, date.day, hour+1, 0)) 
                      for hour in range(9, 17)]
    
    best_time = None
    max_availability = 0
    
    for start, end in possible_times:
        start_iso = start.isoformat() + 'Z'
        end_iso = end.isoformat() + 'Z'
        availability = get_availability(service, attendees, start_iso, end_iso)
        
        available_count = sum(1 for cal in availability.values() if not cal['busy'])
        
        if available_count > max_availability:
            max_availability = available_count
            best_time = (start, end)
    
    return best_time



In [ ]:
def create_event(service, attendees, best_time):
    event = {
        'summary': 'Reunión',
        'location': 'Virtual',
        'description': 'Reunión para discutir el proyecto.',
        'start': {
            'dateTime': best_time[0].isoformat(),
            'timeZone': 'UTC',
        },
        'end': {
            'dateTime': best_time[1].isoformat(),
            'timeZone': 'UTC',
        },
        'attendees': [{'email': attendee} for attendee in attendees],
        'reminders': {
            'useDefault': False,
            'overrides': [
                {'method': 'email', 'minutes': 24 * 60},
                {'method': 'popup', 'minutes': 10},
            ],
        },
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))



In [ ]:
attendees = ['email1@example.com', 'email2@example.com', 'email3@example.com']
date = datetime.datetime(2024, 7, 20)

best_time = find_best_time(service, attendees, date)

if best_time:
    create_event(service, attendees, best_time)
else:
    print("No suitable time found for the meeting.")